# Working with Events

Methods for retrieving open access data.

A database schema diagram for production instances of CDP may be found [here](https://github.com/CouncilDataProject/cdptools/blob/master/docs/resources/database_diagram.pdf).

# Connecting to resources

Having access to both the CDP instance's database and file store with make accessing and using the transcripts easiest. It is recommended to read the database, file store, and transcript usage notebooks prior to working through this one.

For details on database usage, refer to the notebook example on database basics [here](./database.ipynb).

For details on file store usage, refer to the notebook example on file store basics [here](./file_store.ipynb).

**Note:** This notebook connects to the staging instance of Seattle's Firestore database and file store. To use production data, connect to the Cloud Firestore instance: `cdp-seattle`. To use production files, connect to the GCS instance: `cdp-seattle.appspot.com`.

In [1]:
from cdptools.databases.cloud_firestore_database import CloudFirestoreDatabase
from cdptools.file_stores.gcs_file_store import GCSFileStore
import pandas as pd

db = CloudFirestoreDatabase("stg-cdp-seattle")
fs = GCSFileStore("stg-cdp-seattle.appspot.com")
db, fs

(<CloudFirestoreDatabase [stg-cdp-seattle]>,
 <GCSFileStore [stg-cdp-seattle.appspot.com]>)

### Get all votes for a single person

Query and merge the `person` and `vote` tables.

In [2]:
# Filtering not supported for open access API yet, get all people then dataframe filter to target
people = pd.DataFrame(db.select_rows_as_list("person"))
teresa = people.loc[people["full_name"] == "Teresa Mosqueda"]

# Pull voting data
votes = pd.DataFrame(db.select_rows_as_list("vote"))

# Merge data
teresa_voting_record = teresa.merge(
    votes,
    left_on="person_id",
    right_on="person_id",
    suffixes=("_person", "_vote")
)

teresa_voting_record.head()

,created_person,email,full_name,legistar_person_id,person_id,phone,website,created_vote,decision,event_minutes_item_id,legistar_event_item_vote_id,vote_id
0,2019-07-07 23:46:07.899796,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,54535c28-3c38-4b73-ac27-058f3e8af688,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-07 23:46:32.458895,In Favor,9dbac6bd-3775-4994-a204-56225f1d6742,46546,04d63b2e-b486-46ff-b0f0-05847c3403fe
1,2019-07-07 23:46:07.899796,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,54535c28-3c38-4b73-ac27-058f3e8af688,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-07 23:46:49.199693,In Favor,a42fa52d-3dcd-40cc-8c7a-6e84a37ea3ba,46578,06844331-42fd-405c-9927-d9a9ea977311
2,2019-07-07 23:46:07.899796,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,54535c28-3c38-4b73-ac27-058f3e8af688,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-08 01:59:03.139155,In Favor,ff5218bd-752e-416a-9221-6bafdb748e6d,46324,0abd1c88-ad8f-4aff-a409-cd7153bd9234
3,2019-07-07 23:46:07.899796,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,54535c28-3c38-4b73-ac27-058f3e8af688,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-08 01:54:58.521432,In Favor,8dfcb433-2a27-4a5a-bf7a-2bd73c958015,46081,0e610367-5b08-470e-b492-c73671d444df
4,2019-07-07 23:46:07.899796,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,54535c28-3c38-4b73-ac27-058f3e8af688,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-08 01:55:25.031472,In Favor,45d815df-74e4-4f13-8173-85fcc067cb80,46153,11f9eff9-69fe-4c1c-82ee-1671c923e8c7


This is great, but it's missing context of what was actually being voted on. Currently we can see that she voted on instances of `event_minutes_item`, but what were those. Follow the same procedure of `select_rows_as_list` -> `merge` to join the data of the other tables to Teresa's voting record.

In [3]:
# Get event minutes item context data
event_minutes_items = pd.DataFrame(db.select_rows_as_list("event_minutes_item"))
minutes_items = pd.DataFrame(db.select_rows_as_list("minutes_item"))
events = pd.DataFrame(db.select_rows_as_list("event"))

# Merge minutes item data
event_minutes_items = event_minutes_items.merge(
    minutes_items,
    left_on="minutes_item_id",
    right_on="minutes_item_id",
    suffixes=("_event_minutes_item", "_minutes_item")
)

# Merge event data
event_minutes_items = event_minutes_items.merge(
    events,
    left_on="event_id",
    right_on="event_id",
    suffixes=("_event_minutes_item", "_event")
)

# Merge above event minutes item data with Teresa's voting record
teresa_voting_record = teresa_voting_record.merge(
    event_minutes_items,
    left_on="event_minutes_item_id",
    right_on="event_minutes_item_id",
    suffixes=("_voting_record", "_event_minutes_item")
)

# Sort by event datetime
teresa_voting_record = teresa_voting_record.sort_values(by=["event_datetime"])
teresa_voting_record.head()

,created_person,email,full_name,legistar_person_id,person_id,phone,website,created_vote,decision_voting_record,event_minutes_item_id,...,name,agenda_file_uri,body_id,created,event_datetime,legistar_event_id,legistar_event_link,minutes_file_uri,source_uri,video_uri
15,2019-07-07 23:46:07.899796,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,54535c28-3c38-4b73-ac27-058f3e8af688,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-08 01:54:07.183195,In Favor,92e89952-1c70-4fde-8250-90d5478ec2a7,...,"June 10, 2019",http://legistar2.granicus.com/seattle/meetings...,1b73c822-39e7-4252-aedb-4d22de5e4efb,2019-07-08 01:53:56.538108,2019-06-10 14:00:00,3983,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
17,2019-07-07 23:46:07.899796,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,54535c28-3c38-4b73-ac27-058f3e8af688,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-08 01:57:00.944863,In Favor,1ddb7c69-d578-43d7-98b8-8cbaf4c75e1c,...,"Appointment of Shawn Weeks as member, Joint Ap...",http://legistar2.granicus.com/seattle/meetings...,1b73c822-39e7-4252-aedb-4d22de5e4efb,2019-07-08 01:53:56.538108,2019-06-10 14:00:00,3983,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
16,2019-07-07 23:46:07.899796,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,54535c28-3c38-4b73-ac27-058f3e8af688,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-08 01:56:17.745820,In Favor,4f644549-5347-4cfc-8a32-7ad67d31a34f,...,"Appointment of Mikhaila B. Gonzales as member,...",http://legistar2.granicus.com/seattle/meetings...,1b73c822-39e7-4252-aedb-4d22de5e4efb,2019-07-08 01:53:56.538108,2019-06-10 14:00:00,3983,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
22,2019-07-07 23:46:07.899796,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,54535c28-3c38-4b73-ac27-058f3e8af688,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-08 01:54:43.803192,In Favor,704097dd-df23-415b-bbc6-14b950987fcc,...,CB 119538,http://legistar2.granicus.com/seattle/meetings...,1b73c822-39e7-4252-aedb-4d22de5e4efb,2019-07-08 01:53:56.538108,2019-06-10 14:00:00,3983,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
14,2019-07-07 23:46:07.899796,Teresa.Mosqueda@seattle.gov,Teresa Mosqueda,594,54535c28-3c38-4b73-ac27-058f3e8af688,206-684-8806,http://www.seattle.gov/council/mosqueda,2019-07-08 01:57:12.706530,In Favor,10fe03f4-0f23-47c7-bfc4-f5f5a4f59f3c,...,"Reappointment of Thomas J. Kelly as member, Jo...",http://legistar2.granicus.com/seattle/meetings...,1b73c822-39e7-4252-aedb-4d22de5e4efb,2019-07-08 01:53:56.538108,2019-06-10 14:00:00,3983,https://seattle.legistar.com/MeetingDetail.asp...,http://legistar2.granicus.com/seattle/meetings...,http://www.seattlechannel.org/FullCouncil?vide...,http://video.seattle.gov:8080/media/council/co...
